In [1]:
# Import 
import os
import time
import json
import datetime
import pandas as pd
import numpy as np

from pathlib import Path
from tyssue import Sheet
from tyssue.io import hdf5
from tyssue.dynamics.sheet_gradients import height_grad
from tyssue.dynamics import units, effectors, model_factory
from tyssue.solvers.quasistatic import QSSolver
from tyssue.utils import ar_calculation
import matplotlib.pyplot as plt
%matplotlib inline


import os
import time
import json
import random
import datetime
import numpy as np
import pandas as pd

from IPython.display import display

from pathlib import Path
from tyssue import config
from tyssue import Sheet
from tyssue.io import hdf5
from tyssue.io.hdf5 import load_datasets
from tyssue.topology import all_rearangements
from tyssue.core.history import HistoryHdf5
from tyssue.dynamics import SheetModel as basemodel
from tyssue.solvers.quasistatic import QSSolver
from tyssue.draw import sheet_view
from tyssue.draw.plt_draw import quick_edge_draw
from tyssue.draw.ipv_draw import sheet_view as ipv_draw 

from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.apoptosis_events import apoptosis
from tyssue.behaviors.sheet.basic_events import reconnect

from tyssue.utils import to_nd

In [2]:
SIM_DIR = Path('/home/admin-suz/Documents/Simulations/')
today = datetime.date.today().strftime('%Y%m%d')

sim_save_dir = SIM_DIR/f'{today}_TissuPolarization_all_replicat12'

try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [3]:
from polarity.toolbox import define_polarity_old
from polarity.dynamics import EllipsoidLameGeometry as geom
from polarity.delamination import delamination

In [4]:

from polarity.dynamics import model
from polarity.apoptosis import apoptosis, apoptosis_patterning
from polarity.dynamics import EllipsoidLameGeometry as geom
from polarity.toolbox import (init,
                              define_fold_position,
                              apoptosis_ventral,
                              apoptosis_lateral,
                              apoptosis_dorsal,
                              define_polarity
                             )
from tyssue.dynamics import units, effectors, model_factory
from polarity.dynamics import RadialTension, BarrierElasticity, PerimeterElasticity

smodel = model_factory(
    [
        BarrierElasticity,
        RadialTension,
        PerimeterElasticity,
        effectors.FaceAreaElasticity,
        effectors.LumenVolumeElasticity,
    ], effectors.FaceAreaElasticity)


model = smodel

In [5]:
def init():
    dsets = hdf5.load_datasets('../examples/superegg_final.hf5',
                               data_names=['vert', 'edge', 'face'])

    with open('../examples/superegg_final.json', 'r+') as fp:
        specs = json.load(fp)

    sheet = Sheet('spherical', dsets, specs)    
    
    sheet.settings['geometry'] = "spherical"
    sheet.settings['lumen_vol_elasticity'] = 1.e-5
    sheet.settings['barrier_ray'] = 100

    sheet.vert_df['barrier_elasticity'] = 280.0
    sheet.vert_df['delta_rho'] = 0
    
    # A définir en fonction de l'angle de la jonction
    sheet.edge_df['weighted'] = 1.
    sheet.edge_df['weighted_length'] = sheet.edge_df.weighted * sheet.edge_df.length
    
    sheet.face_df['apoptosis'] = 0
    sheet.face_df['current_traction'] = 0.0
    sheet.face_df['radial_tension'] = 0.0
    sheet.face_df['prefered_perimeter'] = 2*np.sqrt(sheet.face_df['prefered_area'])
    sheet.face_df['perimeter_elasticity'] = 10.
    sheet.face_df['area_elasticity'] = 1.
    
    sheet.settings['apopto_pattern_kwargs'] = apopto_pattern_kwargs
    sheet.settings['apoptosis'] = apoptosis_settings
    sheet.settings['rosette_kwargs'] = rosette_kwargs
    sheet.settings['contraction_lt_kwargs'] = contraction_lt_kwargs
    sheet.settings['delaminate_setting'] = delaminate_settings

    geom.update_all(sheet)
    
    return sheet


In [6]:
solver = QSSolver(with_t1=False, with_t3=False)

In [7]:
apopto_pattern_kwargs = {'t':0.,
                      'dt':1.,
                     'time_of_last_apoptosis':30.}
# apoptose
apoptosis_settings = {
        "critical_area_pulling":10,
        "critical_area": 0.5,
        "contract_rate": 1.08,
        "basal_contract_rate": 1.01,
        "contract_neighbors": True,
        "contract_span": 3,
        "radial_tension": 30.,
        "max_traction": 30.,
        "current_traction":0.,
        "geom": geom,
    }

# pour la propagation aux voisins
contraction_lt_kwargs = {
    'face_id':-1,
    'face':-1,
    'shrink_rate':1.05,
    'critical_area': 5.,
    "contraction_column": "line_tension",
    "model":model,
    }

 # Add events to limit rosette 
rosette_kwargs={
    'threshold_length':1e-6,
    'p_4':0.9,
    'p_5p':0.9}

# clone perturbateur
delaminate_settings = {
        'radial_tension':30,
        "contract_rate": 1.08,
        "critical_area_pulling":15,
        "critical_area": 1e-2,
        'current_traction': 0,
        'max_traction': 150,
        'contract_neighbors': True,
        'contract_span': 3,
        'basal_contract_rate': 1.01,
        'geom': geom,
        'model':model}

In [8]:
def run_sim(sim_save_dir, sheet2, constriction, radial_tension, weighted=0):
    
    # without copy, dataframe is on read only... 
    sheet = sheet2.copy()
    
    dirname = '{}_constriction_{}_radialtension_{}_weighted'.format(
        constriction,radial_tension, weighted)
    dirname = os.path.join(sim_save_dir, dirname)
    
    print('starting {}'.format(dirname))
    try:
        os.mkdir(dirname)
    except IOError:
        pass
    
    # Add some information to the sheet and copy initial sheet
    sheet.face_df['id'] = sheet.face_df.index.values

    # Initiate history
    history = HistoryHdf5(sheet, 
                          extra_cols={"face": sheet.face_df.columns,
                                      "edge": list(sheet.edge_df.columns),
                                      "vert": ['x',
                                             'y',
                                             'z',
                                             'is_active',
                                             'rho',
                                             'height',
                                             'basal_shift',
                                             'srce_o',
                                             'barrier_elasticity',
                                             'delta_rho',
                                             'barrier_rho']},
                         hf5file=os.path.join(dirname, 'out.hf5'))
    
    # Initiate manager
    manager = EventManager('face')
    
    # Update kwargs...
    sheet.settings['apoptosis'].update(
                    {
                    'contract_rate':constriction,
                    'radial_tension':float(radial_tension)
                    })
    
    #save settings
    pd.Series(sheet.settings).to_csv(os.path.join(dirname, 'settings.csv'))
   
    
    manager.append(reconnect, **sheet.settings['rosette_kwargs'])
    manager.append(apoptosis_patterning, **sheet.settings['apopto_pattern_kwargs'])

    t = 0.
    stop = 100.
    # Run simulation
    while t < stop:
        
        if t == 5 : 
            for i in sheet.face_df[sheet.face_df.is_mesoderm==1].index:
                delamination_kwargs = sheet.settings['delaminate_setting'].copy()
                delamination_kwargs.update(
                        {
                            "face_id": i,
                            #"radial_tension": radial_tension,
                            "radial_tension":200,
                            "contract_rate": 1.08,
                            "max_traction":90,
                            "current_traction":0,
                        }
                    )
                manager.append(delamination, **delamination_kwargs)
        
               
        
        # Reset radial tension at each time step
        #sheet.vert_df.radial_tension = 0.

        manager.execute(sheet)
        res = solver.find_energy_min(sheet, geom, model, options={"gtol": 1e-8})

        # add noise on vertex position to avoid local minimal.
        sheet.vert_df[
            ['x', 'y']] += np.random.normal(scale=1e-3, size=(sheet.Nv, 2))
        geom.update_all(sheet)
        
        
        # Save result in each time step.
        figname = os.path.join(
            dirname, 'invagination_{:04.0f}.png'.format(t))
        hdfname = figname[:-3] + 'hf5'        
        hdf5.save_datasets(hdfname, sheet)

        history.record(time_stamp=float(t))

        manager.update()
        t += 1.
    
    
    
    print('{} done'.format(dirname))
    print('~~~~~~~~~~~~~~~~~~~~~\n')

## Test several value of polarity ratio

In [11]:
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
import time 
global_start=datetime.now()
#1, 0.9, 0.8, 0.7,0.6, 0.5, 0.4, 0.3, 0.2, 0.1
perpendicular_weighted = [1, 0.9, 0.8, 0.7, 0.6, 0.5]

def modif_polar(weighted):
    time.sleep(np.random.rand())
    sheet=init()
    define_fold_position(sheet, fold_number = 1, position=[-8, 8])
    sheet.face_df.apoptosis = 0
    
    sheet.face_df['is_mesoderm']=0
    sheet.face_df.loc[330, 'is_mesoderm']=1
    
    
    define_polarity_old(sheet, 1, weighted)
    l=[   3,    7,   37,   39,   71,  178,  194,  242,  297,  349,  405,
             464,  488,  502,  653,  881,  892,  938,  985, 1020, 1062, 1075,
            1096, 1118, 1158, 1215, 1275, 1330, 1363, 1374, 1375, 1393, 1437,
            1491, 1543, 1605]
    for i in l :
        sheet.face_df.loc[i, 'apoptosis']=1
    
    run_sim(sim_save_dir, sheet, 1.08, 50, weighted) 
    
"""for pw in perpendicular_weighted:
    modif_polar(pw)"""

results = Parallel(n_jobs=3)(delayed(modif_polar)(
    pw) 
                 for pw in perpendicular_weighted)


global_end = datetime.now()
print ("end : " + str(global_end))
print ('Duree totale d execution : \n\t\t')
print (global_end-global_start)

end : 2019-12-05 05:19:01.525950
Duree totale d execution : 
		
11:12:08.843340


In [ ]:
    l=[   3,    7,   37,   39,   71,  178,  194,  242,  297,  349,  405,
             464,  488,  502,  653,  881,  892,  938,  985, 1020, 1062, 1075,
            1096, 1118, 1158, 1215, 1275, 1330, 1363, 1374, 1375, 1393, 1437,
            1491, 1543, 1605]
    
    =[   4,    7,   37,   39,   72,   86,  192,  254,  297,  424,  471,
             506,  541,  559,  628,  653,  658,  694,  696,  701,  732,  767,
             818,  827,  848,  859,  881,  936,  938,  985, 1020, 1025, 1060,
            1062, 1096, 1125, 1135, 1158, 1174, 1205, 1215, 1393, 1424, 1437,
            1491, 1605]

# Analysis

In [ ]:
"""# Depth fold measurement
from polarity.toolbox import open_sheet,depth_calculation, all_depth_calculation
SIM_DIR = Path('/media/admin-suz/Sophie/2019/Replicat_Fig3/')

sim_save_dir = SIM_DIR/f'20191201_TissuPolarization_all_replicat_0'
list_directory = os.listdir(sim_save_dir)

all_depths= {}
for dir_ in list_directory :
    d_ = all_depth_calculation(os.path.join(sim_save_dir,dir_))
    all_depths[dir_.split('_')[4]] = d_
    

fig, ax = plt.subplots(1, 1, figsize=(20, 10))

for key, value in all_depths.items():
    ax.plot(value, label=key)
    
ax.legend(loc='lower right')
plt.savefig('polarization_depth_time.eps', dpi = 300, transparent=True)"""

In [13]:
def panel_sagittal_view(directory, t= None, ax=None):
    print(directory)
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.get_figure()
        
    if t is None:
        depths = all_depth_calculation(directory)
        t = np.argmin(depths)
    print('temps: ' + str(np.argmin(depths))+ '\tdepth: '+ str(min(depths)))
    sheet = open_sheet(directory, t)

    def draw_specs(sheet):
        specs = {
            'face': {
                'visible': True,
                'color':sheet.face_df.apoptosis + sheet.face_df.fold,
                'colormap':'Blues',
            },
            'edge': {
                'visible': True,
                'color':'black',
            },
            'vert':{
                'visible':False,
                's':100,
                'alpha': 0.3,
                'zorder': 2,
                'color': sheet.vert_df.radial_tension

            }

        }
        return specs
    
    sheet = sheet.extract_bounding_box(z_boundary=[-30,30], y_boundary=(0,100))
    fig, ax = sheet_view(sheet, coords=['z', 'x'], 
                         ax= ax, **draw_specs(sheet))
    ax.grid(False)
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

    #ax.text(1, 2, round(r, 3), horizontalalignment='center',
    #        verticalalignment='center')

In [16]:
# Depth fold measurement
from polarity.toolbox import open_sheet,depth_calculation, all_depth_calculation
SIM_DIR = Path('/media/admin-suz/Sophie/2019/Replicat_Fig3/')

sim_save_dir = SIM_DIR/f'20191204_TissuPolarization_all_replicat1'

import matplotlib
import matplotlib.gridspec as gridspec
dirname =sim_save_dir
fig = plt.figure()
matplotlib.rc('axes', edgecolor='white')
grid = gridspec.GridSpec(1, 6, wspace=0.0, hspace=0.0)
axes_0_0 = fig.add_subplot(grid[0, 0])
axes_0_1 = fig.add_subplot(grid[0, 1])
axes_0_2 = fig.add_subplot(grid[0, 2])
axes_0_3 = fig.add_subplot(grid[0, 3])
axes_0_4 = fig.add_subplot(grid[0, 4])
axes_0_5 = fig.add_subplot(grid[0, 5])


panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_1_weighted/', ax=axes_0_0)
panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_0.9_weighted/', ax=axes_0_1)
panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_0.8_weighted/', ax=axes_0_2)
panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_0.7_weighted/', ax=axes_0_3)
panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_0.6_weighted/', ax=axes_0_4)
panel_sagittal_view(dirname/'1.08_constriction_50_radialtension_0.5_weighted/', ax=axes_0_5)


plt.savefig('polarization_maximal_depth1.eps', dpi = 300, transparent=True)

/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_1_weighted
temps: 58	depth: 0.8370969638259571
/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_0.9_weighted
temps: 60	depth: 0.8236298993715753
/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_0.8_weighted
temps: 58	depth: 0.8000088303579074
/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_0.7_weighted
temps: 59	depth: 0.7772435254430493
/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_0.6_weighted
temps: 59	depth: 0.7646712703102562
/media/admin-suz/Sophie/2019/Replicat_Fig3/20191204_TissuPolarization_all_replicat1/1.08_constriction_50_radialtension_0.5_weighted
temps: 60	depth: 0.73106367637

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

...

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

In [ ]:
"""# polarity measurement

perpendicular_weighted = [1, 0.9,0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

first = 1
for pw in perpendicular_weighted :
    print(pw)
    depths = all_depth_calculation(os.path.join(dirname,('1.08_constriction_50_radialtension_'+str(pw)+'_weighted')))
    t = np.argmin(depths)
    print('temps: ' + str(np.argmin(depths))+ '\tdepth: '+ str(min(depths)))
    sheet = open_sheet(os.path.join(dirname, ('1.08_constriction_50_radialtension_'+str(pw)+'_weighted')), t)
    
    sheet_ = sheet.extract_bounding_box(z_boundary=(-5,5), x_boundary=(20,100))
    anisotropy_fold = ar_calculation(sheet_, ['y','z'])
    
    sheet_ = sheet.extract_bounding_box(z_boundary=(-50,40), x_boundary=(20,100))
    anisotropy_segment = ar_calculation(sheet_, ['y','z'])
    
    if first :
        data_polarity = pd.DataFrame({ pw:anisotropy_fold} )
        data_polarity_segment = pd.DataFrame({ pw:anisotropy_segment} )
        first = 0
    else : 
        data_polarity[pw]=anisotropy_fold
        data_polarity_segment[pw]=anisotropy_segment"""

In [ ]:
"""import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
matplotlib.rc('axes', edgecolor='black')
data = data_polarity

fig = plt.figure(1,figsize=(20, 10))
#create automatically box plot from data_frame
data.boxplot(grid = False, showfliers=False)"""

In [ ]:
"""data_polarity['position']='fold'
data_polarity_segment['position']='segment'"""

In [ ]:
"""frames = [data_polarity, data_polarity_segment]
result = pd.concat(frames)"""

In [ ]:
"""fold = pd.DataFrame(columns=['anisotropy', 'pw', 'position'])
for pw in perpendicular_weighted:
    for v in data_polarity[pw]:
        tmp = {'anisotropy':v,
               'pw':pw,
               'position':'fold'
              }
        fold = fold.append(tmp, ignore_index=True)
        
segment = pd.DataFrame(columns=['anisotropy', 'pw', 'position'])
for pw in perpendicular_weighted:
    for v in data_polarity_segment[pw]:
        tmp = {'anisotropy':v,
               'pw':pw,
               'position':'segment'
              }
        segment = segment.append(tmp, ignore_index=True)
frames = [fold, segment]
result = pd.concat(frames)       """

In [ ]:
"""import seaborn as sns
fig = plt.figure(1,figsize=(20, 10))
sns.boxplot(y='anisotropy', x='pw', 
                 data=result, 
                 palette="colorblind",
                 hue='position', 
                 order= [1, 0.9,0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1],
                 showfliers=False)

plt.savefig('TissuePolarization_fold_segment.eps', dpi = 300, transparent=True)"""